<a href="https://colab.research.google.com/github/adamd1985/Lectures_On_MLAI/blob/main/4_5_SVM_Lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Support Vector Machines (SVM)

In this notebook, we will implement the algorithms taught in the lesson on Support Vector Machines.

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

We use the Iris dataset

In [ ]:
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer(as_frame=True)
breast_cancer.frame.sample(3)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = breast_cancer.data.iloc[:, :2] # first 2 mean texture, mean radius.
y = breast_cancer.target
y = np.where(y == 0, -1, 1)  # Perceptron: -1 is not malignant.

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

## The Perceptron Algorithm

The Perceptron is a simple linear classifier that updates its weights based on misclassified examples. Given a dataset $\mathcal{D} = \{(x^{(n)}, y^{(n)})\}$ where $y^{(n)} \in \{-1, +1\}$ (sometimes used as $v$, induced local field), the model predicts:

$$
\hat{y}^{(n)} = \text{sign}(\beta^\top x^{(n)} + \beta_0)
$$

### Weight Update Rule

If $\hat{y}^{(n)} \neq y^{(n)}$, the weights are updated as:

$$
\beta \leftarrow \beta + \eta y^{(n)} x^{(n)}
$$

$$
\beta_0 \leftarrow \beta_0 + \eta y^{(n)}
$$

where $\eta$ is the learning rate.

If $\hat{y}^{(n)} = y^{(n)}$, no update is performed, meaning:

$$
\beta \leftarrow \beta
$$

$$
\beta_0 \leftarrow \beta_0
$$

### Convergence

If the data is linearly separable, the perceptron is guaranteed to converge to a solution. Otherwise, it oscillates indefinitely.


In [ ]:
class Perceptron:
    def __init__(self, learning_rate=0.01, max_iters=1000):
        self.lr = learning_rate
        self.max_iters = max_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in tqdm(range(self.max_iters)):
            for idx, x_i in enumerate(X):
                if y[idx] * (np.dot(x_i, self.w) + self.b) <= 0:
                    self.w += self.lr * y[idx] * x_i
                    self.b += self.lr * y[idx]

    def predict(self, X):
        return np.sign(np.dot(X, self.w) + self.b)

In [ ]:
from sklearn.metrics import accuracy_score

perceptron = Perceptron(learning_rate=0.01, max_iters=1000)
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

We plot the decision boundaries between 2 features

In [ ]:
def plot_decision_boundary(X, y, model, feature_indices=(0, 1), feature_names=("Feature 1", "Feature 2")):
    X_selected = X[:, feature_indices]

    x_min, x_max = X_selected[:, 0].min() - 1, X_selected[:, 0].max() + 1
    y_min, y_max = X_selected[:, 1].min() - 1, X_selected[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    X_grid = np.zeros((xx.ravel().shape[0], X.shape[1]))
    X_grid[:, feature_indices[0]] = xx.ravel()
    X_grid[:, feature_indices[1]] = yy.ravel()

    Z = model.predict(X_grid)
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.3, levels=np.linspace(Z.min(), Z.max(), 3))
    plt.scatter(X_selected[:, 0], X_selected[:, 1], c=y, edgecolors='k', marker='o', label="Data Points", alpha=0.6)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel(feature_names[0])
    plt.ylabel(feature_names[1])
    plt.title("Decision Boundary")
    plt.legend()
    plt.show()


plot_decision_boundary(X_test, y_test, model=perceptron, feature_names=breast_cancer.feature_names[:2])

That was easy, here the boundary is linearly serperable.

What if we chose a set that are not?

In [ ]:
X = breast_cancer.data.iloc[:, [0,3]]
y = breast_cancer.target
y = np.where(y == 0, -1, 1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

In [ ]:
perceptron = Perceptron(learning_rate=0.01, max_iters=1000)
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

An 83% accuracy with a linear decision boundary suggests that mean texture and mean area are not perfectly linearly separable.

Many data points are very close to the decision boundary, and some misclassified points.

Since the data points follow a curved pattern, a linear model cannot fully capture their relationship.

In [ ]:
plot_decision_boundary(X_test, y_test, model=perceptron, feature_names=(breast_cancer.feature_names[1], breast_cancer.feature_names[3]))

# SVMs

## Linear SVM

Support Vector Machines (SVMs) extend the perceptron by introducing the concept of maximizing margins. Instead of just finding a separating hyperplane, SVM optimizes for the decision boundary with the widest margin between classes. Given a dataset $\mathcal{D} = \{(x^{(n)}, y^{(n)})\}$ with $y^{(n)} \in \{-1,+1\}$, SVM solves:

$$
\min_{\beta, \beta_0} \frac{1}{2} ||\beta||^2
$$

subject to:

$$
y^{(n)} (\beta^\top x^{(n)} + \beta_0) \geq 1, \quad \forall n
$$

This ensures that all points lie outside the margin. The margin is given by:

$$
M = \frac{2}{||\beta||}
$$

To find the optimal decision boundary, we use **subgradient descent** to minimize the **hinge loss** with **L2 regularization**. The **hinge loss** is defined as:

$$
L_{\text{hinge}} = \sum_{n} \max(0, 1 - y^{(n)} (\beta^\top x^{(n)} + \beta_0)) + \lambda ||\beta||^2
$$

The **decision function** determines whether a sample satisfies the margin constraint:

$$
\hat{y}^{(n)} = \text{sign}(\beta^\top x^{(n)} + \beta_0)
$$

- If the current sample **satisfies** the margin constraint $ y^{(n)} \hat{y}^{(n)} \geq 1$:
  $$
  \beta \leftarrow \beta - \eta (2 \lambda \beta)
  $$

- If the current sample **violates** the margin constraint $ y^{(n)} \hat{y}^{(n)} < 1$:
  $$
  \beta \leftarrow \beta - \eta (2 \lambda \beta - y^{(n)} x^{(n)})
  $$
  $$
  \beta_0 \leftarrow \beta_0 - \eta y^{(n)}
  $$

where:
- $ \eta $ is the **learning rate**.
- $ \lambda $ is the **regularization parameter** controlling margin maximization.
- $ \beta $ is the **weight vector**, and $ \beta_0 $ is the **bias**.
- The **decision function** $ \hat{y} = \text{sign}(\beta^\top x + \beta_0) $ is used to determine classification.


In [ ]:
class SupportVectorMachine:
    def __init__(self, learning_rate=0.01, lambda_param=0.01, max_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.max_iters = max_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in tqdm(range(self.max_iters)):
            margins = y * (X @ self.w + self.b)
            misclassified = margins < 1

            grad_w = 2 * self.lambda_param * self.w - np.mean((misclassified * y)[:, None] * X, axis=0)
            grad_b = -np.mean(misclassified * y)

            self.w -= self.lr * grad_w
            self.b -= self.lr * grad_b

    def predict(self, X):
        return np.sign(X @ self.w + self.b)

In [ ]:
X = breast_cancer.data.iloc[:, [0,3]]
y = breast_cancer.target
y = np.where(y == 0, -1, 1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm = SupportVectorMachine(learning_rate=0.01)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
plot_decision_boundary(X_test, y_test, model=svm, feature_names=(breast_cancer.feature_names[1], breast_cancer.feature_names[3]))

## Soft-Margin SVM

Soft-margin SVM allows some misclassification by introducing slack variables $\xi_i$, balancing margin maximization and error minimization. The optimization problem is:

$$
\min_{w, b, \xi} \frac{1}{2} ||w||^2 + C \sum_{i=1}^{n} \xi_i
$$

subject to:

$$
y_i (w^\top x_i + b) \geq 1 - \xi_i, \quad \xi_i \geq 0, \quad \forall i
$$

where $C$ controls the trade-off between maximizing the margin and minimizing classification.

In gradient-based optimization, the weight update incorporates slack terms:

$$
w \leftarrow w - \eta \left( 2\lambda w - C \sum_{i \in \mathcal{M}} y_i x_i \right)
$$

$$
b \leftarrow b - \eta \left( -C \sum_{i \in \mathcal{M}} y_i \right)
$$

where $\mathcal{M} = \{i | y_i (w^\top x_i + b) < 1 \}$ are the misclassified points.

In [ ]:
class SupportVectorMachineSlack:
    def __init__(self, learning_rate=0.01, lambda_param=0.01, C=1.0, max_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.C = C
        self.max_iters = max_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in tqdm(range(self.max_iters)):
            margins = y * (X @ self.w + self.b)
            slack = margins < 1

            grad_w = 2 * self.lambda_param * self.w - self.C * np.mean((slack * y)[:, None] * X, axis=0)
            grad_b = -self.C * np.mean(slack * y)

            self.w -= self.lr * grad_w
            self.b -= self.lr * grad_b

    def predict(self, X):
        return np.sign(X @ self.w + self.b)


In [ ]:
X = breast_cancer.data.iloc[:, [0,3]]
y = breast_cancer.target
y = np.where(y == 0, -1, 1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm = SupportVectorMachineSlack(learning_rate=0.01, C=1.5)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
plot_decision_boundary(X_test, y_test, model=svm, feature_names=(breast_cancer.feature_names[1], breast_cancer.feature_names[3]))

## Dual Formulation of Soft-Margin SVM

Support Vector Machines (SVM) optimize a **Lagrangian formulation** to maximize the margin while allowing some misclassification. The dual form is derived using **Lagrange multipliers**, introducing $\alpha_i$ for each constraint:

$$
\max_{\alpha} \sum_{i=1}^{n} \alpha_i - \frac{1}{2} \sum_{i=1}^{n} \sum_{j=1}^{n} \alpha_i \alpha_j y_i y_j K(x_i, x_j)
$$

subject to:
$$
0 \leq \alpha_i \leq C, \quad \sum_{i=1}^{n} \alpha_i y_i = 0
$$

where:
- $K(x_i, x_j)$ is a general **kernel function**. For a **linear SVM**, $K(x_i, x_j) = x_i^\top x_j$.
- **Soft-margin SVM** introduces slack variables $\xi_i$ in the primal formulation, but they are **absorbed** into the dual through the constraint $0 \leq \alpha_i \leq C$.


Once $\alpha$ is optimized, the **weight vector** for a linear SVM is:
$$
w = \sum_{i=1}^{n} \alpha_i y_i x_i.
$$

The **bias term** is computed using **support vectors with $0 < \alpha_i < C$**:
$$
b = y_i - \sum_{j=1}^{n} \alpha_j y_j K(x_j, x_i), \quad \text{for any } 0 < \alpha_i < C.
$$


Instead of computing $w$ explicitly, predictions are made using the **kernel function**:
$$
f(x) = \sum_{i=1}^{n} \alpha_i y_i K(x_i, x) + b.
$$

## Optimizing Alphas in SVMs with SMO

Support Vector Machines (SVMs) in their **dual formulation** require optimizing **Lagrange multipliers** ($\alpha_i$) while maintaining constraints. To efficiently solve this, we use the **Sequential Minimal Optimization (SMO) algorithm**.

Instead of optimizing **all** $\alpha$ values at once (which is computationally expensive), SMO:
- Selects **two $\alpha$ values at a time** to update.
- Ensures **the sum constraint**:
  $$
  \sum_{i} \alpha_i y_i = 0
  $$
- **Efficiently finds the optimal $\alpha$ values** while keeping constraints valid.


Step 1: Select Two Alphas ($\alpha_i$ and $\alpha_j$)
Instead of updating **all** $\alpha$ values at once, SMO picks **two indices ($i$ and $j$)** and updates only these two.

- Compute **the error for $\alpha_i$**:
  $$
  E_i = \sum_j \alpha_j y_j K(x_i, x_j) + b - y_i
  $$
- Select a **random second index $j$**.

Step 2: Compute Bounds ($L$ and $H$)
The values of $\alpha_i$ and $\alpha_j$ must **stay within a valid range**:

- If $ y_i \neq y_j $, the bounds are:
  $$
  L = \max(0, \alpha_j - \alpha_i)
  $$
  
  $$
  H = \min(C, C + \alpha_j - \alpha_i)
  $$
- If $ y_i = y_j $, the bounds are:
  $$
  L = \max(0, \alpha_i + \alpha_j - C)
  $$
  $$
  H = \min(C, \alpha_i + \alpha_j)
  $$

If $ L = H $, no update is needed.

Step 3: Compute $\eta$ (Step Size)
To determine the **step size** of updates, we compute:

$$
\eta = 2K(x_i, x_j) - K(x_i, x_i) - K(x_j, x_j)
$$

If $ \eta \geq 0 $, we **skip the update** because it would increase the loss.

Step 4: Update $\alpha_j$ and Clip It
Using the SMO formula:

$$
\alpha_j = \alpha_j - \frac{y_j (E_i - E_j)}{\eta}
$$

Then, **clip** $\alpha_j$ to stay within the bounds:

$$
\alpha_j = \min(H, \max(L, \alpha_j))
$$

If the change in $\alpha_j$ is too small, we stop updating.



Step 5: Update $\alpha_i$ to Maintain Constraints
To ensure $ \sum \alpha_i y_i = 0 $, we update $\alpha_i$:

$$
\alpha_i = \alpha_i + y_i y_j (\alpha_j^{\text{old}} - \alpha_j)
$$



Step 6: Update the Bias ($b$)
Since $\alpha$ values changed, we must also **update the bias term**:

$$
b_1 = b - E_i - y_i (\alpha_i - \alpha_i^{\text{old}}) K(x_i, x_i) - y_j (\alpha_j - \alpha_j^{\text{old}}) K(x_i, x_j)
$$

$$
b_2 = b - E_j - y_i (\alpha_i - \alpha_i^{\text{old}}) K(x_i, x_j) - y_j (\alpha_j - \alpha_j^{\text{old}}) K(x_j, x_j)
$$

- If $ 0 < \alpha_i < C $, set $ b = b_1 $.
- Else if $ 0 < \alpha_j < C $, set $ b = b_2 $.
- Otherwise, set $ b = \frac{b_1 + b_2}{2} $.

Step 7: Repeat Until Convergence
- Keep selecting new pairs $(i, j)$ and updating $\alpha$ values.
- Stop when the **change in $\alpha$ is small** (converged).

In [ ]:
class SupportVectorMachineDual:
    def __init__(self, C=1.0, kernel=None, max_iters=1000, tol=1e-5):
        self.C = C
        self.kernel = kernel if kernel else self.linear_kernel
        self.max_iters = max_iters
        self.tol = tol
        self.alpha = None
        self.b = 0
        self.support_vectors = None
        self.support_labels = None
        self.support_alphas = None

    def linear_kernel(self, X1, X2):
        return X1 @ X2.T

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y = y.astype(float)
        K = self.kernel(X, X)

        self.alpha = np.zeros(n_samples)
        self.b = 0

        for _ in tqdm(range(self.max_iters)):
            alpha_prev = np.copy(self.alpha)

            for i in range(n_samples):
                # Compute error for sample i
                E_i = np.dot((self.alpha * y), K[:, i]) + self.b - y[i]
                j = np.random.choice([x for x in range(n_samples) if x != i])
                E_j = np.dot((self.alpha * y), K[:, j]) + self.b - y[j]

                alpha_i_old, alpha_j_old = self.alpha[i], self.alpha[j]

                # Compute L and H (bounds for alpha)
                if y[i] != y[j]:
                    L = max(0, self.alpha[j] - self.alpha[i])
                    H = min(self.C, self.C + self.alpha[j] - self.alpha[i])
                else:
                    L = max(0, self.alpha[i] + self.alpha[j] - self.C)
                    H = min(self.C, self.alpha[i] + self.alpha[j])

                if L == H:
                    continue

                # Compute second derivative of the objective function
                eta = 2 * K[i, j] - K[i, i] - K[j, j]
                if eta >= 0:
                    continue

                # Update alpha_j
                self.alpha[j] -= (y[j] * (E_i - E_j)) / eta
                self.alpha[j] = np.clip(self.alpha[j], L, H)

                if abs(self.alpha[j] - alpha_j_old) < self.tol:
                    continue

                # Update alpha_i to maintain sum constraint
                self.alpha[i] += y[i] * y[j] * (alpha_j_old - self.alpha[j])

                # Compute b (bias term)
                b1 = self.b - E_i - y[i] * (self.alpha[i] - alpha_i_old) * K[i, i] - y[j] * (self.alpha[j] - alpha_j_old) * K[i, j]
                b2 = self.b - E_j - y[i] * (self.alpha[i] - alpha_i_old) * K[i, j] - y[j] * (self.alpha[j] - alpha_j_old) * K[j, j]

                if 0 < self.alpha[i] < self.C:
                    self.b = b1
                elif 0 < self.alpha[j] < self.C:
                    self.b = b2
                else:
                    self.b = (b1 + b2) / 2

            diff = np.linalg.norm(self.alpha - alpha_prev)
            if diff < self.tol:
                break

        support_vector_indices = self.alpha > 1e-6
        self.support_vectors = X[support_vector_indices]
        self.support_labels = y[support_vector_indices]
        self.support_alphas = self.alpha[support_vector_indices]

    def predict(self, X):
        K_test = self.kernel(X, self.support_vectors)
        return np.sign(np.sum(self.support_alphas * self.support_labels * K_test, axis=1) + self.b)

In [ ]:
X = breast_cancer.data.iloc[:, [0,3]]
y = breast_cancer.target
y = np.where(y == 0, -1, 1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm = SupportVectorMachineDual(C=0.5)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
plot_decision_boundary(X_test, y_test, model=svm, feature_names=(breast_cancer.feature_names[1], breast_cancer.feature_names[3]))

## The Kernel Trick in Support Vector Machines (SVM)

Support Vector Machines (SVMs) can handle nonlinear classification problems using the **kernel trick**.


- **Linear Kernel**: Preserves the original feature space.
  
  $$
  K(x, x') = x^\top x'
  $$

- **Polynomial Kernel**: Introduces interactions between features, enabling curved decision boundaries.

  $$
  K(x, x') = (x^\top x' + c)^d
  $$

- **Radial Basis Function (RBF) Kernel**: Maps points into a higher-dimensional space using a Gaussian transformation, allowing flexible decision boundaries.

  $$
  K(x, x') = \exp(-\gamma ||x - x'||^2)
  $$

- **Sigmoid Kernel**: Resembles activation functions in neural networks.

  $$
  K(x, x') = \tanh(\alpha x^\top x' + c)
  $$

Each kernel provides a different way of transforming the data while keeping computations in the dual space

In [ ]:
X = breast_cancer.data.iloc[:, [0,1]]
y = breast_cancer.target
y = np.where(y == 0, -1, 1)


scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def rbf_kernel(X1, X2, gamma=0.5):
    sq_dist = np.linalg.norm(X1[:, np.newaxis] - X2, axis=2) ** 2
    return np.exp(-gamma * sq_dist)


svm_rbf = SupportVectorMachineDual(C=0.8, kernel=lambda X1, X2: rbf_kernel(X1, X2, gamma=3.5))
svm_rbf.fit(X_train, y_train)
y_pred = svm_rbf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


plot_decision_boundary(X_test, y_test, model=svm_rbf, feature_names=(breast_cancer.feature_names[1], breast_cancer.feature_names[3]))

In [ ]:
from sklearn.svm import SVC
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.pipeline import make_pipeline


X = breast_cancer.data.iloc[:, [0, 1]]
y = np.where(breast_cancer.target == 0, -1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def plot_svm_decision_boundary(kernel, X_train, y_train, X_test, y_test, ax, feature_names, class_names):
    svc_clf = make_pipeline(StandardScaler(), SVC(kernel=kernel, C=1))
    svc_clf.fit(X_train, y_train)

    X_train = np.array(X_train)

    y_pred = svc_clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    common_params = {"estimator": svc_clf, "X": X_train, "ax": ax}
    DecisionBoundaryDisplay.from_estimator(
        **common_params,
        response_method="predict",
        plot_method="pcolormesh",
        alpha=0.3,
    )
    DecisionBoundaryDisplay.from_estimator(
        **common_params,
        response_method="decision_function",
        plot_method="contour",
        levels=[-1, 0, 1],
        colors=["k", "k", "k"],
        linestyles=["--", "-", "--"],
    )

    support_vectors = svc_clf.named_steps['standardscaler'].inverse_transform(
        svc_clf.named_steps['svc'].support_vectors_
    )
    ax.scatter(
        support_vectors[:, 0],
        support_vectors[:, 1],
        s=150,
        facecolors="none",
        edgecolors="k",
        label="Support Vectors",
    )

    scatter = ax.scatter(
        X_train[:, 0],
        X_train[:, 1],
        c=y_train,
        cmap=plt.cm.Paired,
        edgecolors="k",
        s=50,
    )

    legend_labels = [class_names[int(label)] for label in np.unique(y_train)]
    ax.legend(
        scatter.legend_elements()[0],
        legend_labels,
        loc="upper right",
        title="Classes",
    )

    ax.set_title(f"{kernel} kernel in SVC\nAccuracy: {accuracy:.2f}\n({feature_names[0]} vs {feature_names[1]})")

kernels = ["linear", "poly", "rbf", "sigmoid"]
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for kernel, ax in zip(kernels, axes.ravel()):
    plot_svm_decision_boundary(kernel, X_train, y_train, X_test, y_test, ax,
                               feature_names=[breast_cancer.feature_names[0], breast_cancer.feature_names[1]],
                               class_names=["Benign", "Malignant"])


# Support Vector Regression (SVR)

Support Vector Regression (SVR) extends Support Vector Machines (SVM) to regression tasks. Instead of finding a maximum margin separator, SVR tries to fit a function $f(\beta)$ that deviates from the true outputs by at most a margin $\epsilon$, while minimizing model complexity. The optimization problem is formulated as:

$$
\min_{\beta, b, \xi, \xi^*} \frac{1}{2} ||\beta||^2 + C \sum_{i=1}^{n} (\xi_i + \xi_i^*)
$$

subject to:

$$
y_i - (\beta^T \beta_i + b) \leq \epsilon + \xi_i
$$

$$
(\beta^T \beta_i + b) - y_i \leq \epsilon + \xi_i^*
$$

$$
\xi_i, \xi_i^* \geq 0
$$

where:

- $\beta$ represents the weight vector,
- $b$ is the bias term,
- $\xi_i, \xi_i^*$ are slack variables that allow violations of the $\epsilon$-tube,
- $C$ controls the trade-off between model complexity and tolerance for outliers,
- $\epsilon$ defines the margin of tolerance.

# Dual Formulation of SVR

In the **dual formulation**, slack variables are not explicitly needed, as they are absorbed into the **Lagrange multipliers**. The **optimization problem** is formulated as:

$$
\max_{\alpha, \alpha_j} \sum_{i=1}^{n} ( \alpha_j - \alpha_i ) y_i - \frac{1}{2} \sum_{i=1}^{n} \sum_{j=1}^{n} ( \alpha_j - \alpha_i )( \alpha_j - \alpha_i ) K(x_i, x_j)
$$

subject to:

$$
0 \leq \alpha_i, \alpha_j \leq C, \quad \sum_{i=1}^{n} (\alpha_i - \alpha_j) = 0
$$

where:
- **$\alpha_i, \alpha_j$** are **Lagrange multipliers**, which replace the slack variables.
- **$ K(x_i, x_j) $** is the **kernel function**, allowing non-linear regression.
- The constraint **$\sum_{i} (\alpha_i - \alpha_j) = 0$** ensures balance between **positive and negative deviations**.

Once the **optimal multipliers** $ \alpha_i, \alpha_j $ are found, the final regression function is:

$$
f(x) = \sum_{i=1}^{n} (\alpha_i - \alpha_j) K(x_i, x) + b
$$

where $ b $ is the bias term computed from **support vectors**.


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, r2_score

class LinearSVR:
    def __init__(self, C=1.0, epsilon=0.1, learning_rate=0.01, max_iter=100):
        self.C = C
        self.epsilon = epsilon
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.w = None
        self.b = 0

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in tqdm(range(self.max_iter)):
            y_pred = self.predict(X)
            dw, db = self._compute_gradients(X, y, y_pred)

            self.w -= self.learning_rate * dw
            self.b -= self.learning_rate * db

        return self

    def _compute_gradients(self, X, y, y_pred):
        n_samples = X.shape[0]
        dw = np.zeros_like(self.w)
        db = 0

        for i in range(n_samples):
            residual = y_pred[i] - y[i]
            if abs(residual) <= self.epsilon:
                continue

            # epsilon-insensitive loss
            if residual > 0:
                dw += X[i]
                db += 1
            else:
                dw -= X[i]
                db -= 1
        dw = dw / n_samples + self.C * self.w
        db = db / n_samples

        return dw, db

    def predict(self, X):
        return np.dot(X, self.w) + self.b

data = fetch_california_housing()
X = data.data[:, :2]
y = data.target

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

svr = LinearSVR(C=1.0, epsilon=0.1, learning_rate=0.01, max_iter=1000)
svr.fit(X_train, y_train)

y_train_pred = svr.predict(X_train)
y_test_pred = svr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"\nSVR MSE: {test_mse:.4f}, R²: {test_r2:.4f}")

In [ ]:
from sklearn.svm import SVR
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, r2_score

data = fetch_california_housing()
X = data.data[:2000, :1]
y = data.target[:2000]

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def plot_svr_regression(kernel, X_train, y_train, X_test, y_test, ax, feature_name):
    svr_clf = SVR(kernel=kernel, C=1.0, epsilon=0.1)
    svr_clf.fit(X_train, y_train)

    X_grid = np.linspace(X_train.min(), X_train.max(), 200).reshape(-1, 1)
    y_pred_grid = svr_clf.predict(X_grid)
    y_pred_test = svr_clf.predict(X_test)

    mse = mean_squared_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)

    ax.scatter(X_train, y_train, color="gray", label="Data")
    ax.plot(X_grid, y_pred_grid, color="red", label=f"{kernel} SVR")
    ax.set_xlabel(feature_name)
    ax.set_ylabel("Target")
    ax.set_title(f"{kernel} SVR Regression\nMSE: {mse:.2f}, R²: {r2:.2f}")
    ax.legend()

kernels = ["linear", "poly", "rbf", "sigmoid"]
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
for kernel, ax in zip(kernels, axes.ravel()):
    plot_svr_regression(kernel, X_train, y_train, X_test, y_test, ax, feature_name=data.feature_names[0])

plt.tight_layout()
plt.show()

# Multi-Class SVMs

Multi class SMVs are a simple extension of the SMV algorithm, which instead of altering the models, it trains multiple models.

We have OvR and OvO, for simplicity, we'll code the OvR variant.

In [ ]:
from sklearn.datasets import load_iris

class OneVsRestSVM:
    def __init__(self, C=1.0, kernel=None, max_iters=1000, tol=1e-5):
        self.C = C
        self.kernel = kernel
        self.max_iters = max_iters
        self.tol = tol
        self.classifiers = []
        self.classes = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        for i in range(n_classes):
            print(f"Training SVM for class {self.classes[i]} vs rest")
            # 1 for current class, -1 for all others
            binary_y = np.where(y == self.classes[i], 1, -1)
            clf = SupportVectorMachineDual(
                C=self.C,
                kernel=self.kernel,
                max_iters=self.max_iters,
                tol=self.tol
            )
            clf.fit(X, binary_y)
            self.classifiers.append(clf)

        return self

    def predict(self, X):
        n_samples = X.shape[0]
        n_classes = len(self.classes)
        decision_scores = np.zeros((n_samples, n_classes))

        for i, clf in enumerate(self.classifiers):
            K_test = clf.kernel(X, clf.support_vectors)
            decision_scores[:, i] = np.sum(
                clf.support_alphas * clf.support_labels * K_test,
                axis=1
            ) + clf.b

        return self.classes[np.argmax(decision_scores, axis=1)]

    def predict_proba(self, X):
        n_samples = X.shape[0]
        n_classes = len(self.classes)
        decision_scores = np.zeros((n_samples, n_classes))

        for i, clf in enumerate(self.classifiers):
            K_test = clf.kernel(X, clf.support_vectors)
            decision_scores[:, i] = np.sum(
                clf.support_alphas * clf.support_labels * K_test,
                axis=1
            ) + clf.b
        exp_scores = np.exp(decision_scores)
        proba = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        return proba

data = load_iris()
X = data.data
y = data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=100, random_state=42)
svc_ovr = OneVsRestSVM(C=0.5)
svc_ovr.fit(X_train, y_train)

y_pred = svc_ovr.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nOne-vs-Rest SVC - Accuracy: {accuracy:.4f}")

Plot the decision boundaries

In [ ]:
def plot_multiclass_decision_boundary(X, y, model, feature_indices=(0, 1), feature_names=("Feature 1", "Feature 2")):
    X_selected = X[:, feature_indices]

    x_min, x_max = X_selected[:, 0].min() - 1, X_selected[:, 0].max() + 1
    y_min, y_max = X_selected[:, 1].min() - 1, X_selected[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    X_grid = np.zeros((xx.ravel().shape[0], X.shape[1]))
    X_grid[:, feature_indices[0]] = xx.ravel()
    X_grid[:, feature_indices[1]] = yy.ravel()

    Z = model.predict(X_grid)
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.3, levels=np.arange(len(np.unique(y)) + 1) - 0.5)
    plt.scatter(X_selected[:, 0], X_selected[:, 1], c=y, edgecolors='k', marker='o', label="Data Points")

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel(feature_names[0])
    plt.ylabel(feature_names[1])
    plt.title("Multiclass Decision Boundary (OvR SVC)")
    plt.legend()
    plt.show()

plot_multiclass_decision_boundary(X_test, y_test, model=svc_ovr, feature_names=data.feature_names[:2])

Compare with Scikit's

In [ ]:
from sklearn.svm import SVC

data = load_iris()
X = data.data
y = data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=100, random_state=42)

svc_sklearn = SVC(C=0.5, kernel="linear", decision_function_shape="ovr")
svc_sklearn.fit(X_train, y_train)

y_pred_sklearn = svc_sklearn.predict(X_test)

accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"One-vs-Rest SVC - Accuracy: {accuracy_sklearn:.4f}")

#Conclusion

In this notebook, we explored Support Vector Machines (SVMs) for both classification and regression, along with their extension to multiclass classification using One-vs-Rest (OvR) and Support Vector Regression (SVR). We implemented linear, polynomial, radial basis function (RBF), and sigmoid kernels, analyzing their impact on decision boundaries and predictions.

We have learned that:

- The linear kernel is effective for linearly separable data and works well in both classification and regression.
- The polynomial kernel introduces feature interactions, allowing for more flexible decision boundaries in multiclass classification.
- The RBF kernel maps data into a high-dimensional space, making it powerful for capturing complex patterns in both SVM and SVR.
- The sigmoid kernel, inspired by neural networks, can model non-traditional decision boundaries but is less commonly applied.
- One-vs-Rest (OvR) extends binary SVMs to multiclass problems by training separate classifiers for each class.
- Support Vector Regression (SVR) applies SVM principles to continuous target prediction, balancing margin control with model complexity.